In [ ]:
import pandas as pd
import numpy as np

# Define the path to our processed data
DATA_FILE = '../data/processed/OAI_tri_modal_cohort_knee_level.parquet'

# Load the master cohort
df = pd.read_parquet(DATA_FILE)

print(f"Successfully loaded data with shape: {df.shape}")
print(df[['ID', 'Knee_Side', 'KL_Grade']].head())

In [ ]:
# --- 1. Reload the Data ---
print("--- 1. Reloading Data ---")
df = pd.read_parquet(DATA_FILE)
print(f"Reloaded data with shape: {df.shape}\n")

# --- 2. Clean Known Date Columns ---
print("--- 2. Cleaning Baseline, TKR, and Death Dates ---")

# Define key date columns
baseline_date_col = 'V00EVDATE'   # Baseline visit
right_tkr_col = 'V99ERKDATE'      # Right Knee TKR
left_tkr_col = 'V99ELKDATE'       # Left Knee TKR
death_date_col = 'V99EDDDATE'     # Date of Death

# Define known "missing" strings
missing_strings = ['.: Missing Form/Incomplete Workbook', '.: Missing'] 

# Pre-clean and convert with explicit formats
for col, fmt in [
    (baseline_date_col, '%m/%d/%Y'), # e.g., 06/03/2005
    (right_tkr_col, '%m/%d/%y'),      # e.g., 04/04/06
    (left_tkr_col, '%m/%d/%y'),       # e.g., 10/28/08
    (death_date_col, '%m/%d/%Y')      # e.g., 11/15/2015
]:
    if col in df.columns:
        df[col] = df[col].replace(missing_strings, np.nan)
        df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce')
        print(f"Column '{col}' NaT (Missing) Count: {df[col].isna().sum()}")

In [ ]:
# --- 3. (Corrected) Build Master Visit-Date Lookup Table ---
import glob
import re

print("--- 3. Building Visit-Date Lookup ---")
DATA_DIR = '../data/OAICompleteData_ASCII'
all_clinical_files = glob.glob(f"{DATA_DIR}/AllClinical*.txt")

visit_data_list = []

# --- Step A: Handle Baseline (Visit 00) ---
# We already have this date in our main 'df'
df_baseline_dates = df[['ID', 'V00EVDATE']].copy()
df_baseline_dates.rename(columns={'V00EVDATE': 'VISIT_DATE'}, inplace=True)
df_baseline_dates['VISIT_NUM'] = '0' # Baseline is visit '0'
visit_data_list.append(df_baseline_dates)

# --- Step B: Loop through all Follow-up Visits (01, 02, ... 14) ---
visit_regex = re.compile(r'AllClinical(\d{2}).txt')

for file_path in all_clinical_files:
    match = visit_regex.search(file_path)
    if not match:
        continue
    
    visit_num_str = match.group(1) # e.g., '01', '08'
    
    # Skip baseline (00), we already did it
    if visit_num_str == '00':
        continue
    
    # This is the corrected column name pattern we discovered
    date_col = f"V{visit_num_str}FVDATE" # e.g., V01FVDATE, V08FVDATE
    
    try:
        # Load *only* the ID and the correct date column
        df_visit = pd.read_csv(file_path, sep='|', on_bad_lines='skip', 
                             usecols=['ID', date_col])
        
        # Clean the date
        df_visit[date_col] = pd.to_datetime(df_visit[date_col], format='%m/%d/%Y', errors='coerce')
        
        # Standardize columns
        df_visit['VISIT_NUM'] = visit_num_str.lstrip('0') # '01' -> '1', '08' -> '8'
        df_visit.rename(columns={date_col: 'VISIT_DATE'}, inplace=True)
        
        visit_data_list.append(df_visit[['ID', 'VISIT_NUM', 'VISIT_DATE']])
        
    except ValueError as e:
        # This will catch any files that *still* don't match (e.g., don't have V...FVDATE)
        print(f"Skipping file {file_path}. Error: {e}")
    except Exception as e:
        print(f"General error on {file_path}: {e}")

# --- Step C: Combine all visit data ---
df_visit_lookup = pd.concat(visit_data_list, ignore_index=True)
df_visit_lookup = df_visit_lookup.dropna(subset=['ID', 'VISIT_NUM', 'VISIT_DATE'])
df_visit_lookup = df_visit_lookup.drop_duplicates(subset=['ID', 'VISIT_NUM'], keep='first')

print(f"\nSUCCESS: Built visit lookup table with {df_visit_lookup.shape[0]} unique (ID, Visit) entries.")
print(df_visit_lookup.head())

In [ ]:
# --- 4. Finalize Survival Logic ---
print("--- 4. Finalizing Survival Logic ---")

# 1. Clean V99RNTCNT (e.g., '14: Year 16' -> '14')
df['LAST_VISIT_NUM'] = df['V99RNTCNT'].astype(str).str.split(':').str[0].str.strip()
print(f"Cleaned 'V99RNTCNT'. Example LAST_VISIT_NUMs: {df['LAST_VISIT_NUM'].unique()[:5]}")

# 2. Merge with our new lookup table to find the date of the last contact
df = pd.merge(
    df,
    df_visit_lookup,
    left_on=['ID', 'LAST_VISIT_NUM'],
    right_on=['ID', 'VISIT_NUM'],
    how='left'
)
df.rename(columns={'VISIT_DATE': 'LAST_CONTACT_DATE'}, inplace=True)
print(f"Merged with lookup. Found {df['LAST_CONTACT_DATE'].notna().sum()} last contact dates.")

# 3. TKR Date (from Cell 2)
df['TKR_Date'] = np.where(df['Knee_Side'] == 1, df[right_tkr_col], df[left_tkr_col])

# 4. Event (from Cell 2)
df['event'] = np.where(df['TKR_Date'].notna(), 1, 0)

# 5. Censor Date is the *earliest* of (Last Contact Date) or (Date of Death)
df['CENSOR_DATE'] = df[['LAST_CONTACT_DATE', death_date_col]].min(axis=1)

# 6. End Date is TKR_Date (if event) or CENSOR_DATE (if no event)
df['end_date'] = df['TKR_Date'].fillna(df['CENSOR_DATE'])

# 7. Time to Event
df['time_to_event'] = (df['end_date'] - df[baseline_date_col]).dt.days

# --- 8. Inspect Final Results ---
print("\n--- 5. Final Survival Variable Inspection ---")
print(f"Total TKR Events: {df['event'].sum()} (out of {df.shape[0]} knees)")

invalid_time = df[df['time_to_event'] < 0].shape[0]
print(f"Knees with invalid (< 0) time: {invalid_time}")

# Drop rows with no time information (no baseline or end date)
original_rows = df.shape[0]
df = df.dropna(subset=['time_to_event'])
df = df[df['time_to_event'] >= 0] # Keep only valid time
print(f"Dropped {original_rows - df.shape[0]} rows with missing/invalid time.")
print(f"Final usable cohort shape: {df.shape}\n")

print("--- Sample of Censored Knees (event=0) ---")
print(df[df['event'] == 0][['ID', 'Knee_Side', 'time_to_event', 'event']].head())
print("\n--- Sample of TKR Knees (event=1) ---")
print(df[df['event'] == 1][['ID', 'Knee_Side', 'time_to_event', 'event']].head())

In [ ]:
# --- 5. Find Baseline Clinical Features ---

# List of keywords for features we want
feature_keywords = [
    'AGE',    # Patient Age
    'SEX',    # Patient Sex
    'BMI',    # Body Mass Index
    'WOM',    # WOMAC Score (e.g., V00WOMTSR for Right Knee)
]

print("--- Searching for Clinical Features ---")
found_features = []
for keyword in feature_keywords:
    # Find columns that contain the keyword
    matches = [col for col in df.columns if keyword in col.upper()]
    if matches:
        print(f"Keyword '{keyword}' found in: {matches}")
        found_features.extend(matches)
    else:
        print(f"Keyword '{keyword}' NOT found.")

# Add our key identifiers and targets
key_cols = [
    'ID', 
    'Knee_Side', 
    'KL_Grade',       # Our baseline image feature
    'time_to_event', 
    'event'
]
final_feature_list = key_cols + list(set(found_features)) # Use set() to remove duplicates

print(f"\n--- Final Selected Columns (Hypothesis) ---")
print(final_feature_list)

# Let's inspect the values for these columns
print("\n--- Inspecting Values ---")
print(df[final_feature_list].head())

# Check for missing values in this subset
print("\n--- Missing Value Counts ---")
print(df[final_feature_list].isna().sum())

In [ ]:
# --- 6. Clean and Select Final Clinical/Survival Features ---

# 1. Define a helper function to clean OAI's messy numeric columns
#    (e.g., '1: Male' -> 1, '71.0' -> 71.0, '.: Missing...' -> NaN)
def clean_numeric_col(series):
    # First, replace all non-numeric "missing" strings with NaN
    series = series.replace(
        ['.: Missing Form/Incomplete Workbook', '.: Missing', '.'], np.nan
    )
    # Second, extract the numeric part (handles '1: Male' -> '1')
    series_str = series.astype(str).str.split(':').str[0].str.strip()
    # Convert to numeric, forcing errors (like 'nan') to NaN
    return pd.to_numeric(series_str, errors='coerce')

# --- 2. Define Our Final Feature Set ---
# These are the *only* columns we need for our baseline model.
key_cols = [
    'ID', 
    'Knee_Side', 
    'time_to_event', 
    'event'
]
image_feature_cols = ['KL_Grade'] # Modality 1
clinical_feature_cols = [
    'V00AGE',    # Baseline Age
    'P02SEX',    # Sex
    'P01BMI',    # Baseline BMI
    'V00WOMTSR', # Baseline WOMAC Total Score (Right)
    'V00WOMTSL'  # Baseline WOMAC Total Score (Left)
]

# 3. Create our final model-ready DataFrame
df_model_data = df[key_cols + image_feature_cols + clinical_feature_cols].copy()
print(f"Created df_model_data with shape: {df_model_data.shape}\n")

# --- 4. Clean and Engineer Features ---
print("Cleaning and engineering features...")

# Clean the simple numeric columns
df_model_data['Age'] = clean_numeric_col(df_model_data['V00AGE'])
df_model_data['BMI'] = clean_numeric_col(df_model_data['P01BMI'])

# Clean Sex ('1: Male', '2: Female')
df_model_data['Sex'] = clean_numeric_col(df_model_data['P02SEX'])

# Clean the two WOMAC score columns
womac_r_clean = clean_numeric_col(df_model_data['V00WOMTSR'])
womac_l_clean = clean_numeric_col(df_model_data['V00WOMTSL'])

# *** Key Engineering Step ***
# Create a single 'WOMAC_Score' column that selects the correct
# score based on the 'Knee_Side' for that row.
df_model_data['WOMAC_Score'] = np.where(
    df_model_data['Knee_Side'] == 1, # If Side is 1 (Right)
    womac_r_clean,                  # Use the Right WOMAC score
    womac_l_clean                   # Else (Side is 2), use the Left WOMAC score
)

# --- 5. Final Cleanup ---
# Drop the raw columns we've now replaced
df_model_data = df_model_data.drop(columns=[
    'V00AGE', 'P02SEX', 'P01BMI', 'V00WOMTSR', 'V00WOMTSL'
])

print("\n--- Final Model-Ready Data Info ---")
df_model_data.info()

print("\n--- Final Missing Value Counts ---")
# This tells us how many rows we'll need to drop or impute
print(df_model_data.isna().sum())

print("\n--- Final Data Head ---")
print(df_model_data.head())

In [ ]:
# --- 7. Handle Missing Data and Save Final Model-Ready File ---

print(f"Original shape: {df_model_data.shape}")

# 1. Drop all rows with any missing values
df_model_final = df_model_data.dropna()

print(f"Shape after dropping NaN rows: {df_model_final.shape}")
print(f"Total rows dropped: {df_model_data.shape[0] - df_model_final.shape[0]}\n")

# 2. Define the final output path
FINAL_OUTPUT_FILE = '../data/processed/OAI_model_ready_data.parquet'

# 3. Save the final file
try:
    df_model_final.to_parquet(FINAL_OUTPUT_FILE, index=False)
    print(f"SUCCESS: Final model-ready data saved to:")
    print(FINAL_OUTPUT_FILE)
    
    # Verify by reloading it
    df_reloaded = pd.read_parquet(FINAL_OUTPUT_FILE)
    print(f"\nVerification: Reloaded file with shape {df_reloaded.shape}")
    print("All missing values handled:")
    print(df_reloaded.isna().sum())

except Exception as e:
    print(f"An error occurred while saving: {e}")